In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import torch
from torch_geometric.data import Data

In [2]:
df = pd.read_csv("data/sider.csv")
df.columns
df.head()
df.shape

(1427, 28)

In [3]:
smi = df["smiles"][500]
mol = Chem.MolFromSmiles(smi)
mol

In [4]:
edges = []
for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    edges.extend([(i,j), (j,i)])
edge_index = list(zip(*edges))

In [5]:
# Atomic number and chirality for atoms
# Bond type and bond direction for bonds
def atom_feature(atom):
    return [atom.GetAtomicNum(),
           atom.GetChiralTag()]
def bond_feature(bond):
    return [bond.GetBondType(),
           bond.GetBondDir()]

In [6]:
node_features = [atom_feature(a) for a in mol.GetAtoms()]
edge_features = [bond_feature(b) for b in mol.GetBonds()]

In [7]:
g = Data(edge_index=torch.LongTensor(edge_index),
         x=torch.FloatTensor(node_features),
         edge_attr=torch.FloatTensor(edge_features),
         smiles=smi,
         mol=mol)

In [8]:
g.x

tensor([[6., 0.],
        [6., 0.],
        [6., 0.],
        [8., 0.],
        [7., 0.],
        [6., 0.],
        [6., 0.],
        [8., 0.],
        [7., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [8., 0.],
        [8., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [7., 0.],
        [6., 0.],
        [8., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [7., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [7., 0.],
        [6., 0.],
        [6., 0.]])

In [9]:
g.edge_attr

tensor([[ 1.,  0.],
        [ 1.,  0.],
        [ 2.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 2.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 2.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [12.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [12.,  0.],
        [ 1.,  0.],
        [ 1.,  0.],
        [12.,  0.],
        [12.,  0.]])

In [10]:
print(len(g.x))
print(len(g.edge_attr))
print(g.edge_index)
# The first row of the tensor contains the origin of the bond, and second the target
print(g.edge_index.shape)

43
50
tensor([[ 0,  1,  1,  2,  2,  3,  2,  4,  4,  5,  5,  6,  6,  7,  6,  8,  8,  9,
          9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 13, 15,  5, 16, 16, 17, 17, 18,
         18, 19, 19, 20, 20, 21, 21, 22,  1, 23, 23, 24, 24, 25, 24, 26, 26, 27,
         27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36,
         36, 37, 37, 38, 38, 39, 29, 40, 40, 41, 40, 42, 14,  1, 13,  4, 12,  8,
         22, 17, 41, 26, 38, 28, 39, 31, 39, 34],
        [ 1,  0,  2,  1,  3,  2,  4,  2,  5,  4,  6,  5,  7,  6,  8,  6,  9,  8,
         10,  9, 11, 10, 12, 11, 13, 12, 14, 13, 15, 13, 16,  5, 17, 16, 18, 17,
         19, 18, 20, 19, 21, 20, 22, 21, 23,  1, 24, 23, 25, 24, 26, 24, 27, 26,
         28, 27, 29, 28, 30, 29, 31, 30, 32, 31, 33, 32, 34, 33, 35, 34, 36, 35,
         37, 36, 38, 37, 39, 38, 40, 29, 41, 40, 42, 40,  1, 14,  4, 13,  8, 12,
         17, 22, 26, 41, 28, 38, 31, 39, 34, 39]])
torch.Size([2, 100])


In [22]:
# Lets see without Torch
import numpy as np
from rdkit import Chem

edge_index = []
edge_features = []
node_features = []

for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    
    # Building edge index
    edge_index.append([i, j])
    edge_index.append([j, i])

    # Extracting bond features
    bond_feats = [
        bond.GetBondType(),
        bond.GetBondDir()  
    ]
    edge_features.extend([bond_feats, bond_feats])  # Adding features for both directions of the edge

for atom in mol.GetAtoms():
    # Extracting node features
    atom_feats = [
        atom.GetAtomicNum(),
        #atom.GetChiralTag()   #For now we do without this; Seems to be 0 anyways for majority of cases
    ]
    node_features.append(atom_feats)

# Convert to numpy arrays
edge_index = (np.array(edge_index).T).astype(np.float32)  
edge_features = np.array(edge_features).astype(np.float32)
node_features = np.array(node_features).astype(np.float32)
print(edge_features)
print(node_features)
print(edge_index)

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 2.  0.]
 [ 2.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 2.  0.]
 [ 2.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 2.  0.]
 [ 2.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [12.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [12.  0.]

In [23]:
from rdkit import Chem
import numpy as np

num_atoms = mol.GetNumAtoms()
# Adjacency matrix
A = np.zeros((num_atoms, num_atoms), dtype=int)

for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    
    
    A[i, j] = 1
    A[j, i] = 1  

np.fill_diagonal(A, 1) # With this we have Ã = A + I
print(A)
print(A.shape)

[[1 1 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 0 1]]
(43, 43)


In [24]:
W = np.random.rand(num_atoms, num_atoms)
output = (A @ node_features).T @ W
np.maximum(0,output).T # Applying ReLU; This is what is getting used for the next iteration?
# I need to actually include edge features, and also update them

array([[427.58974536],
       [446.34849332],
       [435.00021929],
       [419.56714099],
       [491.27470938],
       [480.07364179],
       [484.94833421],
       [454.28184355],
       [440.20969368],
       [424.95922141],
       [417.13434002],
       [476.44949879],
       [508.72217576],
       [500.6214323 ],
       [438.530698  ],
       [447.5276427 ],
       [487.74820726],
       [453.92059157],
       [454.05906822],
       [506.73459083],
       [492.58960497],
       [455.66304291],
       [501.83610923],
       [403.16820044],
       [471.13049361],
       [515.08764982],
       [497.75418747],
       [472.19566363],
       [451.22958267],
       [532.2437875 ],
       [444.67016171],
       [493.0056866 ],
       [395.53686931],
       [472.06360569],
       [419.28237729],
       [395.74634258],
       [535.8030656 ],
       [429.98431763],
       [426.75340409],
       [417.85701077],
       [348.30519003],
       [435.28725036],
       [499.26882792]])

In [34]:
# Perhaps we can actually have the adjacency matrix already including information about the edges
# such that we don't need to actually do the aggregating step (H = X + E) between node features,
# and edge features (as this in matrix form is complicated), and just revert to dot product
# between last H and A containing edge information. We'll also update this A.

from rdkit import Chem
import numpy as np

num_atoms = mol.GetNumAtoms()
# Using only information about bond type for the respetive adjancency matrix
A = np.zeros((num_atoms, num_atoms), dtype=float)

for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    bond_type = bond.GetBondType() 
    
    A[i, j] = bond_type
    A[j, i] = bond_type  

np.fill_diagonal(A, 1) # diagonal with 1's; Ã = A + I
A

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [37]:
W = np.random.rand(num_atoms, num_atoms)
output = (A @ node_features).T @ W
output_1 = np.maximum(0,output).T #Applying ReLU, might want to try sigmoid to place values between 0 and 1
print(output_1)

[[1304.10108576]
 [1583.93644557]
 [1787.86785221]
 [1521.47866917]
 [1745.60734405]
 [1387.6164246 ]
 [1460.43526522]
 [1838.44153607]
 [1638.34407916]
 [1544.84873746]
 [1490.87936072]
 [1383.53843128]
 [1655.02978044]
 [1888.22889839]
 [1358.66153352]
 [1298.72665632]
 [1338.31388803]
 [1532.20692839]
 [1528.63694223]
 [1509.91061038]
 [1341.02822835]
 [1755.02464059]
 [1498.11878833]
 [1736.33127204]
 [1444.74299075]
 [1908.71821868]
 [1596.78100485]
 [1384.56685579]
 [1226.1825147 ]
 [1649.70555758]
 [1190.98834985]
 [1629.15360742]
 [1485.19070398]
 [1214.77540289]
 [1516.50719707]
 [1829.02474559]
 [1395.89518242]
 [1660.39679685]
 [1615.34986335]
 [1436.48134591]
 [1668.08167305]
 [1485.82529537]
 [1989.44070707]]


In [52]:
A2 = A * (output_1 + output_1.T) # The idea is that at each entry of A
# for a certain edge between node i and j, we have information added based on past A
# and the node representations for i and j
# The update of A should be done after the update of H (node representations)
print(A2)
output_2 = np.maximum(0, (A2 @ output_1).T @ W)
print(output_2) #Yeah, need sigmoid ..

[[2608.20217153 2888.03753133    0.         ...    0.
     0.            0.        ]
 [2888.03753133 3167.87289114 3371.80429778 ...    0.
     0.            0.        ]
 [   0.         3371.80429778 3575.73570442 ...    0.
     0.            0.        ]
 ...
 [   0.            0.            0.         ... 3336.16334609
  3153.90696841 3657.52238011]
 [   0.            0.            0.         ... 3153.90696841
  2971.65059074    0.        ]
 [   0.            0.            0.         ... 3657.52238011
     0.         3978.88141413]]
[[1.00729812e+09 1.22739200e+09 1.39806978e+09 1.16435511e+09
  1.35378892e+09 1.06412540e+09 1.13701938e+09 1.43446723e+09
  1.28881448e+09 1.18906559e+09 1.14580834e+09 1.05719425e+09
  1.24961700e+09 1.44158125e+09 1.03509075e+09 9.79346847e+08
  1.03733076e+09 1.18489995e+09 1.16222736e+09 1.13524300e+09
  1.04397662e+09 1.37439820e+09 1.16904889e+09 1.34505563e+09
  1.10074608e+09 1.48802781e+09 1.23822899e+09 1.06820405e+09
  9.46206333e+08 1.2737505